In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import colors
from matplotlib.ticker import PercentFormatter
import random
%matplotlib inline
import re
import warnings

In [2]:
# success rate of gpu's
# what does job termination mean - lose work? means that calculating time is important
#requested jobs used time and terminated -why a buffer
......................................................
#failures by user - who and not completeing - using a lot of resources 
failed jobs - bad users or hardware failure (exitcode)
envoriment side  -- first value user - exitcode, second environment 

'0:9 and 0:15 are the most common exit codes for jobs with some version of the state of "CANCELLED"


SyntaxError: invalid syntax (1537783105.py, line 4)

In [38]:
warnings.filterwarnings(action='ignore', category=FutureWarning)

In [39]:
accre = pd.read_csv('accre-gpu-jobs-2022-v2.csv', on_bad_lines='skip')
accre

,JOBID,ACCOUNT,USER,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,GPUS,PARTITION,EXITCODE,STATE
0,32880657,malanga,arline,18.66M,2-00:00:00,00:13:13,1,2,1,pascal,0:0,COMPLETED
1,32880701,glasshouse,brady,0,05:00:00,00:00:22,1,3,1,maxwell,0:0,COMPLETED
2,32880718,glasshouse,brady,402.32M,05:00:00,00:08:57,1,3,1,maxwell,0:0,COMPLETED
3,32880719,glasshouse,brady,417.62M,20:00:00,05:09:46,1,1,1,pascal,0:0,COMPLETED
4,32880720,glasshouse,brady,450.09M,15:00:00,04:11:10,1,1,1,pascal,0:0,COMPLETED
...,...,...,...,...,...,...,...,...,...,...,...,...
336945,46754520_26,harovin,porsha,0,5-00:00:00,00:07:07,1,6,1,turing,0:0,RUNNING
336946,46754542,malanga,arline,0,2-00:00:00,00:00:00,0,2,1,pascal,0:0,PENDING
336947,46754560,malanga,arline,0,2-00:00:00,00:00:00,0,2,1,pascal,0:0,PENDING
336948,46754598,glasshouse,treva,145.24M,1-00:00:00,00:01:26,1,5,1,turing,0:0,COMPLETED


In [40]:
accre = pd.DataFrame(accre)
accre

,JOBID,ACCOUNT,USER,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,GPUS,PARTITION,EXITCODE,STATE
0,32880657,malanga,arline,18.66M,2-00:00:00,00:13:13,1,2,1,pascal,0:0,COMPLETED
1,32880701,glasshouse,brady,0,05:00:00,00:00:22,1,3,1,maxwell,0:0,COMPLETED
2,32880718,glasshouse,brady,402.32M,05:00:00,00:08:57,1,3,1,maxwell,0:0,COMPLETED
3,32880719,glasshouse,brady,417.62M,20:00:00,05:09:46,1,1,1,pascal,0:0,COMPLETED
4,32880720,glasshouse,brady,450.09M,15:00:00,04:11:10,1,1,1,pascal,0:0,COMPLETED
...,...,...,...,...,...,...,...,...,...,...,...,...
336945,46754520_26,harovin,porsha,0,5-00:00:00,00:07:07,1,6,1,turing,0:0,RUNNING
336946,46754542,malanga,arline,0,2-00:00:00,00:00:00,0,2,1,pascal,0:0,PENDING
336947,46754560,malanga,arline,0,2-00:00:00,00:00:00,0,2,1,pascal,0:0,PENDING
336948,46754598,glasshouse,treva,145.24M,1-00:00:00,00:01:26,1,5,1,turing,0:0,COMPLETED


In [41]:
accre['USEDMEM'] = accre['USEDMEM'].str.replace(r'[a-zA-Z]','')
accre

,JOBID,ACCOUNT,USER,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,GPUS,PARTITION,EXITCODE,STATE
0,32880657,malanga,arline,18.66,2-00:00:00,00:13:13,1,2,1,pascal,0:0,COMPLETED
1,32880701,glasshouse,brady,0,05:00:00,00:00:22,1,3,1,maxwell,0:0,COMPLETED
2,32880718,glasshouse,brady,402.32,05:00:00,00:08:57,1,3,1,maxwell,0:0,COMPLETED
3,32880719,glasshouse,brady,417.62,20:00:00,05:09:46,1,1,1,pascal,0:0,COMPLETED
4,32880720,glasshouse,brady,450.09,15:00:00,04:11:10,1,1,1,pascal,0:0,COMPLETED
...,...,...,...,...,...,...,...,...,...,...,...,...
336945,46754520_26,harovin,porsha,0,5-00:00:00,00:07:07,1,6,1,turing,0:0,RUNNING
336946,46754542,malanga,arline,0,2-00:00:00,00:00:00,0,2,1,pascal,0:0,PENDING
336947,46754560,malanga,arline,0,2-00:00:00,00:00:00,0,2,1,pascal,0:0,PENDING
336948,46754598,glasshouse,treva,145.24,1-00:00:00,00:01:26,1,5,1,turing,0:0,COMPLETED


In [42]:
accre['REQTIME'] = accre['REQTIME'].astype(str)
accre['USEDTIME'] = accre['USEDTIME'].astype(str)
accre['REQTIME'] = pd.to_timedelta(accre['REQTIME'].str.replace('-',' days '))
accre['USEDTIME'] = pd.to_timedelta(accre['USEDTIME'].str.replace('-',' days '))
accre['REQTIME'] = pd.to_timedelta(accre['REQTIME']).dt.total_seconds()
accre['USEDTIME'] = pd.to_timedelta(accre['USEDTIME']).dt.total_seconds()
accre = accre[accre['USEDMEM'].map(len) > 1]
accre

,JOBID,ACCOUNT,USER,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,GPUS,PARTITION,EXITCODE,STATE
0,32880657,malanga,arline,18.66,172800.0,793.0,1,2,1,pascal,0:0,COMPLETED
2,32880718,glasshouse,brady,402.32,18000.0,537.0,1,3,1,maxwell,0:0,COMPLETED
3,32880719,glasshouse,brady,417.62,72000.0,18586.0,1,1,1,pascal,0:0,COMPLETED
4,32880720,glasshouse,brady,450.09,54000.0,15070.0,1,1,1,pascal,0:0,COMPLETED
5,32880721,glasshouse,brady,403.40,302400.0,67776.0,1,1,1,pascal,0:0,COMPLETED
...,...,...,...,...,...,...,...,...,...,...,...,...
336933,46753813,harovin,shannon,137.96,432000.0,52.0,1,1,1,turing,0:0,COMPLETED
336934,46753820_26,harovin,porsha,5093.70,432000.0,2091.0,1,6,1,turing,0:0,COMPLETED
336936,46753823,harovin,shannon,156.21,432000.0,243.0,1,1,1,turing,0:0,COMPLETED
336942,46754095,harovin,shannon,2820.52,432000.0,673.0,1,1,1,turing,0:0,COMPLETED


In [44]:
accre_df = accre
accre_df

,JOBID,ACCOUNT,USER,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,GPUS,PARTITION,EXITCODE,STATE
0,32880657,malanga,arline,18.66,172800.0,793.0,1,2,1,pascal,0:0,COMPLETED
2,32880718,glasshouse,brady,402.32,18000.0,537.0,1,3,1,maxwell,0:0,COMPLETED
3,32880719,glasshouse,brady,417.62,72000.0,18586.0,1,1,1,pascal,0:0,COMPLETED
4,32880720,glasshouse,brady,450.09,54000.0,15070.0,1,1,1,pascal,0:0,COMPLETED
5,32880721,glasshouse,brady,403.40,302400.0,67776.0,1,1,1,pascal,0:0,COMPLETED
...,...,...,...,...,...,...,...,...,...,...,...,...
336933,46753813,harovin,shannon,137.96,432000.0,52.0,1,1,1,turing,0:0,COMPLETED
336934,46753820_26,harovin,porsha,5093.70,432000.0,2091.0,1,6,1,turing,0:0,COMPLETED
336936,46753823,harovin,shannon,156.21,432000.0,243.0,1,1,1,turing,0:0,COMPLETED
336942,46754095,harovin,shannon,2820.52,432000.0,673.0,1,1,1,turing,0:0,COMPLETED


In [ ]:
#Looking at Accounts Failed rate/distribution/Users from accounts-is there a individual who is failing/
#By what percent are they off/  

In [55]:
accre_index = accre_df.set_index(['ACCOUNT', 'USER', 'USEDMEM', 'REQTIME', 'USEDTIME', 'EXITCODE', 'STATE']).sort_index()
accre_index

JOBID  \
ACCOUNT USER     USEDMEM REQTIME  USEDTIME EXITCODE STATE                 
almonds annetta  0.00    259200.0 17697.0  0:0      COMPLETED  39373374   
                 0.54    14400.0  506.0    0:0      COMPLETED  40921510   
                 12.53   259200.0 6.0      0:0      COMPLETED  39424118   
                 161.09  172800.0 208.0    0:0      COMPLETED  39343851   
                 166.81  86400.0  326.0    0:0      COMPLETED  39342056   
...                                                                 ...   
white   brittani 851.40  432000.0 77334.0  0:0      COMPLETED  41842346   
                 909.12  43200.0  43208.0  0:0      COMPLETED  42143179   
                 913.52  3600.0   2202.0   0:0      COMPLETED  45012538   
                 985.37  43200.0  6863.0   0:0      COMPLETED  41367314   
                 993.08  432000.0 432021.0 0:0      COMPLETED  41867882   

                                                               NODES  CPUS  \
ACCOUNT USER     USEDMEM REQTIME  USEDTIME EXITCODE STATE                    
almonds annetta  0.00    259200.0 17697.0  0:0      COMPLETED      1     6   
                 0.54    14400.0  506.0    0:0      COMPLETED      1     2   
                 12.53   259200.0 6.0      0:0      COMPLETED      1     6   
                 161.09  172800.0 208.0    0:0      COMPLETED      1     5   
                 166.81  86400.0  326.0    0:0      COMPLETED      1     5   
...                                                              ...   ...   
white   brittani 851.40  432000.0 77334.0  0:0      COMPLETED      1    12   
                 909.12  43200.0  43208.0  0:0      COMPLETED      1     9   
                 913.52  3600.0   2202.0   0:0      COMPLETED      1     3   
                 985.37  43200.0  6863.0   0:0      COMPLETED      1    12   
                 993.08  432000.0 432021.0 0:0      COMPLETED      1    12   

                                                               GPUS PARTITION  
ACCOUNT USER     USEDMEM REQTIME  USEDTIME EXITCODE STATE                      
almonds annetta  0.00    259200.0 17697.0  0:0      COMPLETED     1    turing  
                 0.54    14400.0  506.0    0:0      COMPLETED     1    turing  
                 12.53   259200.0 6.0      0:0      COMPLETED     1    turing  
                 161.09  172800.0 208.0    0:0      COMPLETED     1    turing  
                 166.81  86400.0  326.0    0:0      COMPLETED     1    turing  
...                                                             ...       ...  
white   brittani 851.40  432000.0 77334.0  0:0      COMPLETED     4   maxwell  
                 909.12  43200.0  43208.0  0:0      COMPLETED     3   maxwell  
                 913.52  3600.0   2202.0   0:0      COMPLETED     1   maxwell  
                 985.37  43200.0  6863.0   0:0      COMPLETED     4   maxwell  
                 993.08  432000.0 432021.0 0:0      COMPLETED     4   maxwell  

[278200 rows x 5 columns]

In [69]:
accre_index.value_counts('USER', 'ACCOUNT')

USER
marnie      0.505841
margret     0.211499
collins     0.047779
hillery     0.030439
arline      0.029145
              ...   
hattie      0.000004
jefferey    0.000004
pamelia     0.000004
porfirio    0.000004
corry       0.000004
Length: 173, dtype: float64

In [10]:
accre_pascal_state

,JOBID,ACCOUNT,USER,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,GPUS,PARTITION,EXITCODE,STATE
1247,32896784,glasshouse,miya,19.59,00:30:00,00:09:54,1,1,1,pascal,127:0,FAILED
9370,32940928,glasshouse,arlyne,16331.96,1-06:00:00,1-06:00:16,7,7,32,pascal,1:0,FAILED
14105,33016633,sunburst,demarcus,0,4-00:00:00,00:00:15,1,1,1,pascal,1:0,FAILED
14117,33016701,sunburst,demarcus,0,4-00:00:00,00:00:01,1,1,1,pascal,1:0,FAILED
14119,33016901,sunburst,demarcus,0,4-00:00:00,00:00:02,1,1,1,pascal,1:0,FAILED
...,...,...,...,...,...,...,...,...,...,...,...,...
332515,46179924,glasshouse,lea,52415.96,5-00:00:00,3-01:57:43,1,7,8,pascal,1:0,FAILED
332517,46179927,glasshouse,lea,13839.69,5-00:00:00,3-01:54:36,1,7,8,pascal,1:0,FAILED
332746,46287083,glasshouse,tommie,82628.63,3-18:00:00,00:25:12,2,8,8,pascal,1:0,FAILED
334706,46525496,glasshouse,mel,11823.83,5-00:00:00,2-02:00:25,3,12,16,pascal,1:0,FAILED


In [8]:
accre_pascal = accre_df[accre_df['PARTITION'] == 'pascal']
accre_maxwell = accre_df[accre_df['PARTITION'] == 'maxwell']
accre_turing = accre_df[accre_df['PARTITION'] == 'turing']

In [ ]:
fig, ax = plt.subplots(1, 3,figsize = (20,13))


sns.distplot(pascal_usedmem, hist=True, bins = 10, color = 'orange', ax = ax[0]),
sns.distplot(pascal_usedtime, hist=True, bins = 10, color = 'black', ax = ax[0]),
sns.distplot(maxwell_usedmem, hist=False, bins = 5, color = 'orange', ax = ax[1]),
sns.distplot(maxwell_usedtime, hist=False, bins = 5, color = 'black', ax = ax[1]),
sns.distplot(turing_usedmem, hist=False, bins = 5, color = 'orange', ax = ax[2]),
sns.distplot(turing_usedtime, hist=False, bins = 5, color = 'black', ax = ax[2])

plt.show()

In [ ]:
asking for enough memory/types of failures
asking too much memory

In [ ]:
# .squeeze() converts df to series
#accre_list = pd.DataFrame(accre, columns=['USEDMEM'])
#accre_list = accre_list.squeeze()
